# Model Comparison: All Approaches

Train and compare all models on the same data:
1. Single best feature (z-scored)
2. Linear regression (all features)
3. Ridge regression (L2 regularization)
4. Lasso regression (L1 regularization)
5. Random Forest classifier

Compare on:
- Out-of-sample accuracy/correlation
- Overfitting gap (train - test)
- Sharpe ratio from backtesting
- Win rate
- Profit factor

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error
from scipy.stats import pearsonr
import joblib

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

In [ ]:
API_KEY = "vFDjkUVRfPnedLrbRjm75BZ9CJHz3dfv"
TICKER = "AAPL"
START_DATE = "2025-10-01"
END_DATE = "2025-11-01"

In [ ]:
def pull_polygon_data(ticker, start, end, api_key):
    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{start}/{end}?apiKey={api_key}"
    response = requests.get(url)
    data = response.json()
    
    if 'results' not in data or len(data['results']) < 2:
        raise ValueError("not enough data")
    
    df = pd.DataFrame(data['results'])
    df['timestamp'] = pd.to_datetime(df['t'], unit='ms')
    df = df.rename(columns={'o':'open','h':'high','l':'low','c':'close','v':'volume'})
    df = df[['timestamp','open','high','low','close','volume']]
    return df

def calculate_features(df):
    df = df.copy()
    
    df['momentum_1min'] = df['close'].pct_change()
    df['volatility_1min'] = df['momentum_1min'] ** 2
    df['price_direction'] = (df['close'] > df['open']).astype(int)
    df['vwap'] = (df['close'] * df['volume']).cumsum() / df['volume'].cumsum()
    df['vwap_dev'] = (df['close'] - df['vwap']) / df['vwap']
    df['hour'] = df['timestamp'].dt.hour
    df['minute'] = df['timestamp'].dt.minute
    
    df['next_return'] = df['close'].shift(-1) / df['close'] - 1
    df['target_binary'] = (df['next_return'] > 0).astype(int)
    df['target_continuous'] = df['next_return']
    
    df = df.dropna()
    return df

In [ ]:
# load data
df = pull_polygon_data(TICKER, START_DATE, END_DATE, API_KEY)
df = calculate_features(df)

features = ['momentum_1min', 'volatility_1min', 'price_direction', 'vwap_dev', 'hour', 'minute']
X = df[features]
y_binary = df['target_binary']
y_continuous = df['target_continuous']

# chronological split
split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train_bin, y_test_bin = y_binary.iloc[:split_idx], y_binary.iloc[split_idx:]
y_train_cont, y_test_cont = y_continuous.iloc[:split_idx], y_continuous.iloc[split_idx:]

print(f"loaded {len(df)} bars")
print(f"train: {len(X_train)}, test: {len(X_test)}")

## 1. Single Best Feature Model

In [ ]:
# find best feature by IC
ic_scores = {feat: abs(pearsonr(X_train[feat], y_train_cont)[0]) for feat in features}
best_feature = max(ic_scores, key=ic_scores.get)
print(f"best feature: {best_feature}")

# z-score and train
scaler_single = StandardScaler()
X_train_single = scaler_single.fit_transform(X_train[[best_feature]])
X_test_single = scaler_single.transform(X_test[[best_feature]])

model_single = LinearRegression()
model_single.fit(X_train_single, y_train_cont)

pred_train_single = model_single.predict(X_train_single)
pred_test_single = model_single.predict(X_test_single)

# convert to binary for accuracy
pred_train_single_bin = (pred_train_single > 0).astype(int)
pred_test_single_bin = (pred_test_single > 0).astype(int)

single_train_corr = pearsonr(y_train_cont, pred_train_single)[0]
single_test_corr = pearsonr(y_test_cont, pred_test_single)[0]
single_train_acc = accuracy_score(y_train_bin, pred_train_single_bin)
single_test_acc = accuracy_score(y_test_bin, pred_test_single_bin)

## 2. Multi-Feature Linear Regression

In [ ]:
scaler_multi = StandardScaler()
X_train_scaled = scaler_multi.fit_transform(X_train)
X_test_scaled = scaler_multi.transform(X_test)

model_linear = LinearRegression()
model_linear.fit(X_train_scaled, y_train_cont)

pred_train_linear = model_linear.predict(X_train_scaled)
pred_test_linear = model_linear.predict(X_test_scaled)

pred_train_linear_bin = (pred_train_linear > 0).astype(int)
pred_test_linear_bin = (pred_test_linear > 0).astype(int)

linear_train_corr = pearsonr(y_train_cont, pred_train_linear)[0]
linear_test_corr = pearsonr(y_test_cont, pred_test_linear)[0]
linear_train_acc = accuracy_score(y_train_bin, pred_train_linear_bin)
linear_test_acc = accuracy_score(y_test_bin, pred_test_linear_bin)

## 3. Ridge Regression

In [ ]:
model_ridge = Ridge(alpha=0.1)
model_ridge.fit(X_train_scaled, y_train_cont)

pred_train_ridge = model_ridge.predict(X_train_scaled)
pred_test_ridge = model_ridge.predict(X_test_scaled)

pred_train_ridge_bin = (pred_train_ridge > 0).astype(int)
pred_test_ridge_bin = (pred_test_ridge > 0).astype(int)

ridge_train_corr = pearsonr(y_train_cont, pred_train_ridge)[0]
ridge_test_corr = pearsonr(y_test_cont, pred_test_ridge)[0]
ridge_train_acc = accuracy_score(y_train_bin, pred_train_ridge_bin)
ridge_test_acc = accuracy_score(y_test_bin, pred_test_ridge_bin)

## 4. Lasso Regression

In [ ]:
model_lasso = Lasso(alpha=0.0001, max_iter=10000)
model_lasso.fit(X_train_scaled, y_train_cont)

pred_train_lasso = model_lasso.predict(X_train_scaled)
pred_test_lasso = model_lasso.predict(X_test_scaled)

pred_train_lasso_bin = (pred_train_lasso > 0).astype(int)
pred_test_lasso_bin = (pred_test_lasso > 0).astype(int)

lasso_train_corr = pearsonr(y_train_cont, pred_train_lasso)[0]
lasso_test_corr = pearsonr(y_test_cont, pred_test_lasso)[0]
lasso_train_acc = accuracy_score(y_train_bin, pred_train_lasso_bin)
lasso_test_acc = accuracy_score(y_test_bin, pred_test_lasso_bin)

## 5. Random Forest Classifier

In [ ]:
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train_bin)

pred_train_rf = model_rf.predict(X_train)
pred_test_rf = model_rf.predict(X_test)

prob_train_rf = model_rf.predict_proba(X_train)[:, 1]
prob_test_rf = model_rf.predict_proba(X_test)[:, 1]

rf_train_acc = accuracy_score(y_train_bin, pred_train_rf)
rf_test_acc = accuracy_score(y_test_bin, pred_test_rf)
rf_train_auc = roc_auc_score(y_train_bin, prob_train_rf)
rf_test_auc = roc_auc_score(y_test_bin, prob_test_rf)

## Performance Comparison Table

In [ ]:
comparison = pd.DataFrame([
    {
        'model': f'Single Feature ({best_feature})',
        'type': 'regression',
        'train_corr': single_train_corr,
        'test_corr': single_test_corr,
        'train_acc': single_train_acc,
        'test_acc': single_test_acc,
        'corr_gap': single_train_corr - single_test_corr,
        'acc_gap': single_train_acc - single_test_acc
    },
    {
        'model': 'Linear Regression',
        'type': 'regression',
        'train_corr': linear_train_corr,
        'test_corr': linear_test_corr,
        'train_acc': linear_train_acc,
        'test_acc': linear_test_acc,
        'corr_gap': linear_train_corr - linear_test_corr,
        'acc_gap': linear_train_acc - linear_test_acc
    },
    {
        'model': 'Ridge (α=0.1)',
        'type': 'regression',
        'train_corr': ridge_train_corr,
        'test_corr': ridge_test_corr,
        'train_acc': ridge_train_acc,
        'test_acc': ridge_test_acc,
        'corr_gap': ridge_train_corr - ridge_test_corr,
        'acc_gap': ridge_train_acc - ridge_test_acc
    },
    {
        'model': 'Lasso (α=0.0001)',
        'type': 'regression',
        'train_corr': lasso_train_corr,
        'test_corr': lasso_test_corr,
        'train_acc': lasso_train_acc,
        'test_acc': lasso_test_acc,
        'corr_gap': lasso_train_corr - lasso_test_corr,
        'acc_gap': lasso_train_acc - lasso_test_acc
    },
    {
        'model': 'Random Forest',
        'type': 'classifier',
        'train_corr': rf_train_auc,
        'test_corr': rf_test_auc,
        'train_acc': rf_train_acc,
        'test_acc': rf_test_acc,
        'corr_gap': rf_train_auc - rf_test_auc,
        'acc_gap': rf_train_acc - rf_test_acc
    }
])

print("\n=== MODEL COMPARISON ===")
print("\nNote: For regression models, 'corr' = Pearson correlation. For RF, 'corr' = AUC.")
print(comparison[['model', 'test_corr', 'test_acc', 'corr_gap', 'acc_gap']].to_string(index=False))

## Visual Comparison

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# test accuracy comparison
axes[0].barh(comparison['model'], comparison['test_acc'], color='steelblue')
axes[0].set_xlabel('Test Accuracy')
axes[0].set_title('Out-of-Sample Accuracy Comparison')
axes[0].axvline(x=0.5, color='red', linestyle='--', alpha=0.5, label='Random')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# overfitting gap comparison
colors = ['green' if gap < 0.05 else 'orange' if gap < 0.1 else 'red' for gap in comparison['acc_gap']]
axes[1].barh(comparison['model'], comparison['acc_gap'], color=colors)
axes[1].set_xlabel('Overfitting Gap (train_acc - test_acc)')
axes[1].set_title('Overfitting Analysis')
axes[1].axvline(x=0, color='black', linestyle='--', alpha=0.5)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Backtesting Comparison

In [ ]:
def simple_backtest(signals, returns, transaction_cost=0.0001):
    """Simple backtest given signals and actual returns"""
    signals = np.array(signals)
    returns = np.array(returns)
    
    # calculate strategy returns
    position_changes = np.diff(signals, prepend=0)
    strategy_returns = signals * returns - np.abs(position_changes) * transaction_cost
    
    # metrics
    total_return = (1 + strategy_returns).prod() - 1
    sharpe = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252 * 390) if strategy_returns.std() > 0 else 0
    
    trades = strategy_returns[strategy_returns != 0]
    win_rate = (trades > 0).sum() / len(trades) if len(trades) > 0 else 0
    
    winning = trades[trades > 0]
    losing = trades[trades < 0]
    profit_factor = winning.sum() / abs(losing.sum()) if len(losing) > 0 and losing.sum() != 0 else np.inf
    
    return {
        'total_return': total_return,
        'sharpe': sharpe,
        'win_rate': win_rate,
        'profit_factor': profit_factor,
        'num_trades': len(trades)
    }

# convert predictions to signals (1 = long, -1 = short, 0 = flat)
# for regression models, use threshold of 0
signal_single = np.where(pred_test_single > 0, 1, -1)
signal_linear = np.where(pred_test_linear > 0, 1, -1)
signal_ridge = np.where(pred_test_ridge > 0, 1, -1)
signal_lasso = np.where(pred_test_lasso > 0, 1, -1)

# for RF, use probability threshold 0.55
signal_rf = np.where(prob_test_rf > 0.55, 1, np.where(prob_test_rf < 0.45, -1, 0))

# run backtests
bt_single = simple_backtest(signal_single, y_test_cont)
bt_linear = simple_backtest(signal_linear, y_test_cont)
bt_ridge = simple_backtest(signal_ridge, y_test_cont)
bt_lasso = simple_backtest(signal_lasso, y_test_cont)
bt_rf = simple_backtest(signal_rf, y_test_cont)

# buy and hold benchmark
bt_bh = simple_backtest(np.ones(len(y_test_cont)), y_test_cont)

backtest_comparison = pd.DataFrame([
    {'model': f'Single ({best_feature})', **bt_single},
    {'model': 'Linear', **bt_linear},
    {'model': 'Ridge', **bt_ridge},
    {'model': 'Lasso', **bt_lasso},
    {'model': 'Random Forest', **bt_rf},
    {'model': 'Buy & Hold', **bt_bh}
])

print("\n=== BACKTEST COMPARISON ===")
print(backtest_comparison.to_string(index=False))

## Summary and Recommendation

In [ ]:
print("\n=== SUMMARY ===")
print("\nBest Test Accuracy:")
best_acc_idx = comparison['test_acc'].idxmax()
print(f"  {comparison.loc[best_acc_idx, 'model']}: {comparison.loc[best_acc_idx, 'test_acc']:.4f}")

print("\nLowest Overfitting:")
best_gap_idx = comparison['acc_gap'].idxmin()
print(f"  {comparison.loc[best_gap_idx, 'model']}: gap = {comparison.loc[best_gap_idx, 'acc_gap']:.4f}")

print("\nBest Sharpe Ratio:")
best_sharpe_idx = backtest_comparison['sharpe'].idxmax()
print(f"  {backtest_comparison.loc[best_sharpe_idx, 'model']}: {backtest_comparison.loc[best_sharpe_idx, 'sharpe']:.2f}")

print("\nBest Total Return:")
best_return_idx = backtest_comparison['total_return'].idxmax()
print(f"  {backtest_comparison.loc[best_return_idx, 'model']}: {backtest_comparison.loc[best_return_idx, 'total_return']*100:.2f}%")